In [1]:
# https://coderzcolumn.com/tutorials/artificial-intelligence/object-detection-using-pre-trained-pytorch-models

# Get Needed Packages

In [2]:
!pip install torch
!pip install pycocotools

import torch
import random
import pycocotools
import numpy as np
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms.functional import pil_to_tensor
from torchvision.models.detection import fasterrcnn_resnet50_fpn

print("TorchVision Version : {}".format(torchvision.__version__))
print("PyTorch Version : {}".format(torch.__version__))


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


TorchVision Version : 0.13.1+cpu
PyTorch Version : 1.12.1+cpu


# Image Classification Function

## Constants from "get_image_data()" to speed it up

In [3]:
# The actual detection model
object_detection_model = fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
object_detection_model.eval(); ## Setting Model for Evaluation/Prediction

# File with the possible results
annFile='annotations/instances_val2017.json'
coco=COCO(annFile)    

C:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading annotations into memory...
Done (t=1.46s)
creating index...
index created!


In [4]:
# Get the results of the image classification
def get_image_data(path):
    image = Image.open(path)
    image_tensor_int = pil_to_tensor(image)
    image_tensor_int = image_tensor_int.unsqueeze(dim=0)
    image_tensor_float = image_tensor_int / 255.0
    image_preds = object_detection_model(image_tensor_float)
    
    image_preds[0]["boxes"] = image_preds[0]["boxes"][image_preds[0]["scores"] > 0.8]
    image_preds[0]["labels"] = image_preds[0]["labels"][image_preds[0]["scores"] > 0.8]
    image_preds[0]["scores"] = image_preds[0]["scores"][image_preds[0]["scores"] > 0.8]
    
    image_labels = coco.loadCats(image_preds[0]["labels"].numpy())
    
    image_annot_labels = ["{}-{:.2f}".format(label["name"], prob) for label, prob in zip(image_labels, image_preds[0]["scores"].detach().numpy())]
        
    return [image_labels[0]["name"], image_preds[0]["scores"][0].item()]

# Auxiliary Functions

In [5]:
# Change a random pixel of an image to a random color
def change_random_pixel(path):
    image = Image.open(path)
    pixels = image.load()

    width, height = image.size
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    pixels[random.randint(0, width-1), random.randint(0, height-1)] = color
    image.save(path)

In [6]:
# Change a random pixel and analyze the result
def change_and_analyze(path):
    change_random_pixel(path)
    results = get_image_data(path)
    return get_image_data(path)

In [7]:
# Change a set pixel to a random color
def change_pixel_to_random_color(path, pixel_x, pixel_y):
    image = Image.open(path)
    pixels = image.load()

    width, height = image.size
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    pixels[pixel_x, pixel_y] = color
    image.save(path)

In [ ]:
# Change a set pixel to the color of another image
def change_pixel_to_image(path, replacement_path, pixel_x, pixel_y):
    image = Image.open(path)
    pixels = image.load()
    
    replacement_image = Image.open(replacement_path)
    replacement_pixels = replacement_image.load()

    width, height = image.size
    
    pixels[pixel_x, pixel_y] = replacement_pixels[pixel_x, pixel_y]
    image.save(path)

In [8]:
# Clone the paths
def clone_path(base_path, clone_path):
    image = Image.open(base_path)
    image.save(clone_path)

# Get Initial Results

## Inputs

In [ ]:
# Path of the original image
path = "dog.jpg"

# Path of the image to replace, leave as empty string if you want random pixels
replacement_path = ""

# How much you want the accuracy of the image classification to drop
acceptable_accuracy_drop = 0.05

In [10]:
results = []

# Deal with original image
path_2 = f"{2}_{path[:-4]}.png"
    
clone_path(path, path_2)

image = Image.open(path)
pixels = image.load()
width, height = image.size

# Deal with replacement image
replacement_path_2 = ""

if replacement_path != "":
    replacement_path_2 = f"{2}_{replacement_path[:-4]}.png"
    img = Image.open(replacement_path)
    img_resize = img.resize((width, height), Image.LANCZOS)
    img_resize.save(replacement_path_2)

# Get starting variables
base_result_percentage = round(get_image_data(path_2)[1],5)
changes_list = []
changes = 0

# Iterate over image while changing pixels
for x in range(width):
    for y in range(height):
        
        # Setup temp image
        temp_path = f"{3}_{path[:-4]}.png"
        clone_path(path_2, temp_path)
        changes_list.append(changes)
        
        # Change pixels based on random pixels or image
        if replacement_path != "":
            change_pixel_to_image(temp_path, replacement_path_2, pixel_x, pixel_y)
        else:
            change_pixel_to_random_color(temp_path, x, y)
        
        # Result of the changes
        temp_result_percentage = round(get_image_data(path_2)[1],5)
        
        # Print out the information
        print("Iteration{}/{}\tChanges: {}\tWidth:{}/{}\tHeight:{}/{}Base: {}\tTemp: {}\tDifference is {}".format(
            x+(x*y),
            x*y
            changes,
            x, width,
            y, width,
            base_result_percentage,
            temp_result_percentage,
            abs(temp_result_percentage - base_result_percentage)
        ))
        
        # Check error bounds on image
        if temp_result_percentage >= (base_result_percentage-((acceptable_accuracy_drop*x*y)/(width*height))):
            # If within error bounds, keep the image
            clone_path(temp_path, path_2)
            changes += 1
            break
        else:
            # Discard image
            clone_path(path_2, temp_path)
        
        # Update results
        results.append(changes)

Changes: 0	Width:0/400	Height:0/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 1	Width:0/400	Height:1/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 2	Width:0/400	Height:2/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 3	Width:0/400	Height:3/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 4	Width:0/400	Height:4/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 5	Width:0/400	Height:5/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 6	Width:0/400	Height:6/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 7	Width:0/400	Height:7/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 8	Width:0/400	Height:8/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 9	Width:0/400	Height:9/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 10	Width:0/400	Height:10/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 11	Width:0/400	Height:11/400
Base: 0.97893	Temp: 0.97893	Difference is 0.0
Changes: 12	

Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:84/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:85/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:86/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:87/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:88/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:89/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:90/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:91/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:92/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Heig

Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:165/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:166/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:167/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:168/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:169/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:170/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:171/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:172/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:173/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0

Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:246/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:247/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:248/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:249/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:250/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:251/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:252/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:253/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:254/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0

Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:327/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:328/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:329/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:330/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:331/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:332/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:333/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:334/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0/400	Height:335/400
Base: 0.97893	Temp: 0.97892	Difference is 9.99999999995449e-06
Changes: 12	Width:0

...

Base: 0.97893	Temp: 0.98581	Difference is 0.006879999999999997
Changes: 10370	Width:47/400	Height:395/400
Base: 0.97893	Temp: 0.98584	Difference is 0.006910000000000083
Changes: 10371	Width:47/400	Height:396/400
Base: 0.97893	Temp: 0.98584	Difference is 0.006910000000000083
Changes: 10372	Width:47/400	Height:397/400
Base: 0.97893	Temp: 0.98585	Difference is 0.006920000000000037
Changes: 10373	Width:47/400	Height:398/400
Base: 0.97893	Temp: 0.9859	Difference is 0.006970000000000032
Changes: 10374	Width:47/400	Height:399/400
Base: 0.97893	Temp: 0.98591	Difference is 0.006979999999999986
Changes: 10375	Width:47/400	Height:400/400
Base: 0.97893	Temp: 0.98584	Difference is 0.006910000000000083
Changes: 10376	Width:47/400	Height:401/400
Base: 0.97893	Temp: 0.98588	Difference is 0.006950000000000012
Changes: 10377	Width:47/400	Height:402/400
Base: 0.97893	Temp: 0.98589	Difference is 0.006960000000000077
Changes: 10378	Width:47/400	Height:403/400
Base: 0.97893	Temp: 0.98589	Difference is 0.006

Base: 0.97893	Temp: 0.98572	Difference is 0.006790000000000074
Changes: 10448	Width:48/400	Height:21/400
Base: 0.97893	Temp: 0.98572	Difference is 0.006790000000000074
Changes: 10449	Width:48/400	Height:22/400
Base: 0.97893	Temp: 0.98576	Difference is 0.006830000000000003
Changes: 10450	Width:48/400	Height:23/400
Base: 0.97893	Temp: 0.98576	Difference is 0.006830000000000003
Changes: 10451	Width:48/400	Height:24/400
Base: 0.97893	Temp: 0.98575	Difference is 0.006820000000000048
Changes: 10452	Width:48/400	Height:25/400
Base: 0.97893	Temp: 0.98571	Difference is 0.006780000000000008
Changes: 10453	Width:48/400	Height:26/400
Base: 0.97893	Temp: 0.98576	Difference is 0.006830000000000003
Changes: 10454	Width:48/400	Height:27/400
Base: 0.97893	Temp: 0.98579	Difference is 0.006860000000000088
Changes: 10455	Width:48/400	Height:28/400
Base: 0.97893	Temp: 0.9858	Difference is 0.006870000000000043
Changes: 10456	Width:48/400	Height:29/400
Base: 0.97893	Temp: 0.98579	Difference is 0.006860000000

Base: 0.97893	Temp: 0.98559	Difference is 0.006659999999999999
Changes: 10527	Width:48/400	Height:100/400
Base: 0.97893	Temp: 0.98564	Difference is 0.006709999999999994
Changes: 10528	Width:48/400	Height:101/400
Base: 0.97893	Temp: 0.98563	Difference is 0.006700000000000039
Changes: 10529	Width:48/400	Height:102/400
Base: 0.97893	Temp: 0.98571	Difference is 0.006780000000000008
Changes: 10530	Width:48/400	Height:103/400
Base: 0.97893	Temp: 0.98567	Difference is 0.006740000000000079
Changes: 10531	Width:48/400	Height:104/400
Base: 0.97893	Temp: 0.98567	Difference is 0.006740000000000079
Changes: 10532	Width:48/400	Height:105/400
Base: 0.97893	Temp: 0.98564	Difference is 0.006709999999999994
Changes: 10533	Width:48/400	Height:106/400
Base: 0.97893	Temp: 0.9856	Difference is 0.006670000000000065
Changes: 10534	Width:48/400	Height:107/400
Base: 0.97893	Temp: 0.98558	Difference is 0.006650000000000045
Changes: 10535	Width:48/400	Height:108/400
Base: 0.97893	Temp: 0.98554	Difference is 0.006

Base: 0.97893	Temp: 0.98567	Difference is 0.006740000000000079
Changes: 10605	Width:48/400	Height:178/400
Base: 0.97893	Temp: 0.98569	Difference is 0.006759999999999988
Changes: 10606	Width:48/400	Height:179/400
Base: 0.97893	Temp: 0.98568	Difference is 0.006750000000000034
Changes: 10607	Width:48/400	Height:180/400
Base: 0.97893	Temp: 0.98566	Difference is 0.006730000000000014
Changes: 10608	Width:48/400	Height:181/400
Base: 0.97893	Temp: 0.98558	Difference is 0.006650000000000045
Changes: 10609	Width:48/400	Height:182/400
Base: 0.97893	Temp: 0.98556	Difference is 0.006630000000000025
Changes: 10610	Width:48/400	Height:183/400
Base: 0.97893	Temp: 0.98552	Difference is 0.006589999999999985
Changes: 10611	Width:48/400	Height:184/400
Base: 0.97893	Temp: 0.9855	Difference is 0.006570000000000076
Changes: 10612	Width:48/400	Height:185/400
Base: 0.97893	Temp: 0.9855	Difference is 0.006570000000000076
Changes: 10613	Width:48/400	Height:186/400
Base: 0.97893	Temp: 0.98552	Difference is 0.0065

KeyboardInterrupt: 

# Results

## Plot Results

In [ ]:
results_x = [i for i in range(len(results))]

plt.plot(results_x, results)
plt.show()

## Show Results

In [ ]:
# Original Image
original = Image.open(path)
original.show()

In [ ]:
# Finished Image
finished = Image.open(path_2)
finished.show()